In [1]:
import pretty_midi
import numpy as np
import mir_eval.display
import librosa.display
import matplotlib.pyplot as plt
import vamp
import tempfile
import glob
import os
import sox
import csv
import jams
from IPython.display import Audio
%matplotlib inline

In [2]:
def save_as_jams(outname, audiopath, adjusted_onsets):
    jam = jams.JAMS()
    jam.file_metadata.duration = sox.file_info.duration(audiopath)
    ann = jams.Annotation(
        namespace='onset', time=0,
        duration=jam.file_metadata.duration
    )
    ann.annotation_metadata.data_source = str(outname.split('_')[-1])
    for onset_time in adjusted_onsets:
        ann.append(time=onset_time, duration=0)
    jam.annotations.append(ann)
    jam.save(outname+'_onset.jams')
    return jam

def csv_to_jams(outname, audiopath):
    #save pt.csv as jams
    jam = jams.JAMS()
    jam.file_metadata.duration = sox.file_info.duration(audiopath)
    
    ann_raw_pt = jams.Annotation(
        namespace='pitch_contour', time=0,
        duration=jam.file_metadata.duration
    )
    ann_raw_pt.annotation_metadata.data_source = str(os.path.basename(outname).split('_')[0])
    
    with open(outname+'_pt.csv', 'r') as pt:
        reader = csv.reader(pt)
        for row in reader:
            ann_raw_pt.append(time=float(row[0]), duration=0, confidence=None,
                       value={'voiced': True if row[1] else False, 'index': 0, 'frequency': float(row[1])})
    
    ann_pt = jams.Annotation(
        namespace='pitch_contour', time=0,
        duration=jam.file_metadata.duration
    )
    ann_pt.annotation_metadata.data_source = str(os.path.basename(outname).split('_')[0])
    ann_note = jams.Annotation(
        namespace='note_midi', time=0,
        duration=jam.file_metadata.duration
    )
    ann_note.annotation_metadata.data_source = str(os.path.basename(outname).split('_')[0])
    
    with open(outname+'_onoff.csv', 'r') as onoff:
        reader = csv.reader(onoff)
        for on_time, off_time in reader:
            on_time = float(on_time)
            off_time = float(off_time)
            if on_time >= off_time:
                continue
            ann_pt_seg = ann_raw_pt.trim(on_time, off_time)
            for obs in ann_pt_seg:
                ann_pt.append(time=obs.time, duration=obs.duration, value=obs.value, confidence=obs.confidence)
            pitch_midi = get_pitch(ann_pt_seg)
#             print(pitch_hz)
            if pitch_midi == 0:
                continue
#             pitch = librosa.hz_to_midi(pitch_hz)
            ann_note.append(time=on_time, duration=off_time-on_time, value=pitch_midi, confidence=None)
            
        
    jam.annotations.append(ann_pt)       
    jam.annotations.append(ann_note)            
    jam.save(outname+'.jams')
    return jam

def get_pitch(ann, verbose=False):
    avg_pitch = 0
    count = 0
    time_freq = []
    for obs in ann.data:
        if obs.value['voiced'] and float(obs.value['frequency']) > 30: # if its voiced
            time_freq.append([obs.time, float(obs.value['frequency'])])
            count += 1
    
    time_freq = np.asarray(time_freq)
    if verbose:
        plt.plot(time_freq.T[0], librosa.hz_to_midi(time_freq.T[1]))
    
    if count < 4:
        if verbose:
            plt.show()
        return 0
    else:
        low_idx = int(count/4)
        high_idx = int(count/2)
        avg_pitch = np.mean(librosa.hz_to_midi(time_freq.T[1, low_idx:high_idx]))
#         print(avg_pitch)
        if verbose:
            plt.hlines(avg_pitch, time_freq.T[0,0], time_freq.T[0,-1])
            plt.show()
        return avg_pitch
    
    
def build_jams(ordered_jam_list, on, off, piece, outdir):
    new_jam = jams.JAMS()
    new_jam.file_metadata.duration = off-on
    outname = os.path.join(outdir, piece+'.jams')
    
    for jam in ordered_jam_list:
        ann_pt = jam.search(namespace="pitch_contour")[0].slice(on, off)
        ann_note = jam.search(namespace="note_midi")[0].slice(on, off)
        new_jam.annotations.append(ann_pt)
        new_jam.annotations.append(ann_note)
        
    new_jam.save(outname)

In [3]:
audiopaths = glob.glob('./Bounces/*.wav')
audiopaths.sort()
print(audiopaths)

['./Bounces/0_c_1.wav', './Bounces/1_c_1.wav', './Bounces/2_c_1.wav', './Bounces/3_c_1.wav', './Bounces/4_c_1.wav', './Bounces/5_c_1.wav']


In [4]:
all_jam = []
for audiopath in audiopaths:
#     audiopath = audiopaths[1]
    print("new audio file!")
    fname = audiopath.split('/')[-1].split('.')[0]
    outname = os.path.join('./post_tony',fname)
    all_jam.append(csv_to_jams(outname, audiopath))

new audio file!
new audio file!
new audio file!
new audio file!
new audio file!
new audio file!


In [5]:
with open('cutting_times.csv', 'r') as onoff:
    i = 0
    reader = csv.reader(onoff)
    for on, off, piece in reader:
        print(i)
        i += 1
        on = float(on)
        off = float(off)
        piece += '_comp'
        build_jams(all_jam, on, off, piece, './jams')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
